In [1]:
import json
import os
import gzip
from pprint import pprint
from collections import defaultdict
import pandas as pd


In [2]:
def get_subject_properties(person_statements_file, subject_properties):
    with gzip.open(person_statements_file) as f:
        for line in f:
            entry = json.loads(line)
            subject_properties[entry["id"]] = {"income_class": entry["income_class"], "country_id": entry["country_id"], "gender": entry["gender"]}
    return subject_properties

#subject_properties = {}
#subject_properties = get_subject_properties(os.path.join("..", "data", "person_statements", "person_statements_part1.jsonl.gz"), subject_properties)
#subject_properties = get_subject_properties(os.path.join("..", "data", "person_statements", "person_statements_part2.jsonl.gz"), subject_properties)

In [3]:
with open(os.path.join("..", "data", "10k_incomeClass_pred.json")) as f:
    count = 1
    for line in f:
        if count > 0 :
            data = json.loads(line)
            print(data.keys())
            count -= 1
        else:
            break

dict_keys(['Subject', 'Predicate', 'Object', 'SubjectID', 'PredicateID', 'ObjectID', 'URL', 'IncomeClass', 'BirthYear', 'RefDoc', 'claim', 'has_ref', 'evidence', 'certain_claim', 'history_claims', 'ask_questions', 'question_answers', 'sub_claims', 'verify_results', 'prediction', 'model_output'])


In [4]:
def get_prediction_counts(predictions_file):
    by_income = defaultdict(lambda: defaultdict(int))
    by_income_lists = defaultdict(list)
    by_country = {}
    by_gender = {}
    prediction_labels = ["Model Error", "supports", "undecided", "refutes"]
    value_map = dict(zip(prediction_labels, [i for i in range(len(prediction_labels))]))

    with open(predictions_file) as f:
        for line in f:
            data = json.loads(line)
            try:
                by_income[data["IncomeClass"]][data["prediction"]] += 1
                by_income_lists[data["IncomeClass"]] += [value_map[data["prediction"]]]
            except KeyError:
                by_income[data["IncomeClass"]]["None"] += 1
    return by_income, by_income_lists

predictions_by_income, predictions_by_income_lists = get_prediction_counts(os.path.join("..", "data", "10k_incomeClass_pred.json"))


In [5]:
predictions_by_income_lists.keys()

dict_keys(['high_income', 'low_income', 'lower-middle_income', 'upper-middle_income'])

In [6]:
pprint(predictions_by_income)

defaultdict(<function get_prediction_counts.<locals>.<lambda> at 0x11f753dc0>,
            {'high_income': defaultdict(<class 'int'>,
                                        {'Model Error': 1364,
                                         'refutes': 100,
                                         'supports': 759,
                                         'undecided': 277}),
             'low_income': defaultdict(<class 'int'>,
                                       {'Model Error': 1379,
                                        'refutes': 101,
                                        'supports': 730,
                                        'undecided': 290}),
             'lower-middle_income': defaultdict(<class 'int'>,
                                                {'Model Error': 1504,
                                                 'refutes': 81,
                                                 'supports': 701,
                                                 'undecided': 214}),
        

In [10]:
predictions_by_income_df = pd.DataFrame.from_dict(predictions_by_income)
predictions_by_income_df = predictions_by_income_df.loc[predictions_by_income_df.index != "Model Error", :]
predictions_by_income_df = predictions_by_income_df[["low_income", "lower-middle_income", "upper-middle_income", "high_income"]]
predictions_by_income_df["total"] = predictions_by_income_df.sum(axis=1)
predictions_by_income_df["lower_income"] = predictions_by_income_df[["low_income", "lower-middle_income"]].sum(axis=1)
predictions_by_income_df["higher_income"] = predictions_by_income_df[["high_income", "upper-middle_income"]].sum(axis=1)
predictions_by_income_df.div(predictions_by_income_df.sum(axis=0), axis=1)

,low_income,lower-middle_income,upper-middle_income,high_income,total,lower_income,higher_income
supports,0.651204,0.703815,0.564315,0.668134,0.643786,0.675957,0.614695
undecided,0.258698,0.214859,0.322822,0.243838,0.262450,0.238073,0.284494
refutes,0.090098,0.081325,0.112863,0.088028,0.093764,0.085971,0.100812
